In [1]:
import os
import gzip
import glob
import re
import pandas as pd
import numpy as np
from collections import Counter, defaultdict, OrderedDict
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
os.chdir("../../data/10famsim")

In [3]:
def file_iter(file_path):
    sequences = {}
    with gzip.open(file_path, 'rt') as file:
        first_char = file.read(1)
    
    if first_char == '>':
        # Jika file FASTA
        with gzip.open(file_path, 'rt') as file:
            sequence_id = None
            current_sequence = ''
            for line in file:
                if line.startswith('>'):
                    if sequence_id is not None:
                        sequences[sequence_id] = current_sequence
                    sequence_id = line.strip().split(' ')[0]
                    current_sequence = ''
                else:
                    current_sequence += line.strip()
            if sequence_id is not None:
                sequences[sequence_id] = current_sequence
    elif first_char == '@':
        # Jika file FASTQ
        with gzip.open(file_path, 'rt') as file:
            line_count = 0
            sequence_id = None
            current_sequence = ''
            for line in file:
                line_count += 1
                if line_count % 4 == 1:
                    sequence_id = line.strip().split(' ')[0]  # Ambil sequence ID
                elif line_count % 4 == 2:
                    current_sequence = line.strip()  # Urutan nukleotida
                    sequences[sequence_id] = current_sequence
    return sequences

def extract_spaced_kmers(sequence, pattern):
    kmer_length = len(pattern)
    kmers = []
    for i in range(len(sequence) - kmer_length + 1):
        kmer = ''.join(sequence[i + j] if pattern[j] == '1' else '*' for j in range(kmer_length))
        if all(base in 'ATGC*' for base in kmer):
            kmers.append(kmer)
    return kmers

def process_sequence(sequence_id, sequence, patterns):
    all_kmers = []
    for pattern in patterns:
        all_kmers.extend(extract_spaced_kmers(sequence, pattern))
    return sequence_id, Counter(all_kmers)

def count_spaced_kmers(sequences, patterns, n_jobs=8):
    kmer_counts = Parallel(n_jobs=n_jobs)(
        delayed(process_sequence)(sequence_id, sequence, patterns)
        for sequence_id, sequence in tqdm(sequences.items())
    )
    return dict(kmer_counts)

def build_feature_space(kmer_counts, sequences):
    all_kmers = set()
    for kmer_count in kmer_counts.values():
        all_kmers.update(kmer_count.keys())

    all_kmers = sorted(all_kmers)
    feature_space = defaultdict(list)

    for sequence_id, kmer_count in kmer_counts.items():
        sequence_length = len(sequences[sequence_id])
        feature_vector = [(kmer_count.get(kmer, 0) / sequence_length) for kmer in all_kmers]
        feature_space[sequence_id] = feature_vector
    
    return all_kmers, feature_space

In [4]:
# # Fungsi untuk menghasilkan pola spaced k-mers acak
# def generate_random_spaced_kmers():
#     patterns = []
#     for _ in range(1):  # Assuming we create 20 initial patterns
#         pattern_3 = '1' + ''.join(random.choice('01') for _ in range(1)) + '1'
#         pattern_4 = '1' + ''.join(random.choice('01') for _ in range(2)) + '1'
#         pattern_5 = '1' + '0' + ''.join(random.choice('01') for _ in range(2)) + '1'
#         patterns.append([pattern_3, pattern_4, pattern_5])
#     return patterns

# Contigs

## Feature

In [5]:
# Mendapatkan semua file di dalam folder
folder_path = './'
file_paths = glob.glob(os.path.join(folder_path, '*.fna.gz'))

patterns = [['111', '1111', '11111']]
all_features = []
file_names = []  # List untuk menyimpan nama file

for file_path in file_paths:
    print(f"Memproses file: {file_path}")  # Cetak nama file yang sedang diproses
    sequences = file_iter(file_path)
    for pattern_group in patterns:
        kmer_counts = count_spaced_kmers(sequences, pattern_group)
        feature_name, feature_space = build_feature_space(kmer_counts, sequences)
        features = np.array(list(feature_space.values()))
        all_features.append(features)
        file_names.extend([file_path] * len(features))  # Tambahkan nama file untuk setiap sequence

# Gabungkan semua fitur menjadi satu DataFrame
all_features_combined = np.vstack(all_features)
feature_df = pd.DataFrame(all_features_combined)

# Menampilkan DataFrame hasil
feature_df

Memproses file: ./GCF_000009905-Exact.d324fd4b.fna.gz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 3005.75it/s]


Memproses file: ./GCF_000018325-Exact.d3251aed.fna.gz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 3272.70it/s]


Memproses file: ./GCF_000009045-Exact.d324f740.fna.gz


...

Memproses file: ./GCF_000013425-Exact.d324f178.fna.gz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 3260.75it/s]


Memproses file: ./GCF_000012985-Exact.d32508a2.fna.gz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 3297.49it/s]


,0,1,2,3,4,5,6,7,8,9,...,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343
0,0.010,0.002,0.000,0.002,0.000,0.000,0.002,0.000,0.002,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.004,0.002,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.006,0.000,0.000,0.000,0.000,0.000,0.002,0.002,0.000,0.000,...,0.004,0.002,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.002,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.018,0.002,0.000,0.002,0.000,0.000,0.004,0.002,0.002,0.000,...,0.008,0.004,0.000,0.004,0.000,0.000,0.000,0.000,0.000,0.000
99996,0.016,0.004,0.000,0.000,0.002,0.002,0.004,0.002,0.002,0.000,...,0.010,0.004,0.002,0.004,0.000,0.008,0.000,0.002,0.006,0.000
99997,0.036,0.018,0.006,0.000,0.006,0.006,0.002,0.000,0.000,0.002,...,0.008,0.002,0.000,0.000,0.006,0.008,0.004,0.002,0.000,0.002
99998,0.012,0.002,0.000,0.000,0.000,0.002,0.004,0.000,0.000,0.002,...,0.006,0.000,0.002,0.002,0.002,0.004,0.000,0.004,0.000,0.000


In [6]:
# label_path = "./contigs/gsa_mapping.tsv.gz"
# label_df = pd.read_csv(label_path, sep='\t')
# label_df = label_df[["genome_id"]]
# label_df

def fasta_iter(fasta_file):
    with gzip.open(fasta_file, 'rt') as file:
        name = None
        seqs = []
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if name:
                    yield name, ''.join(seqs)
                name = line[1:]  # Skip '>'
                seqs = []
            else:
                seqs.append(line)
        if name:
            yield name, ''.join(seqs)  # Yield the last entry
            
# Function to detect file type based on file extension
def detect_file_type(file_path):
    ext = os.path.splitext(file_path)[-1].lower()  # Get file extension and convert to lowercase
    # Handle gzipped files
    if ext == '.gz':
        ext = os.path.splitext(file_path[:-3])[-1].lower()  # Check the extension before ".gz"
        
    if ext in ['.fasta', '.fa', '.fna']:
        return 'fasta'
    elif ext in ['.fastq', '.fq']:
        return 'fastq'
    else:
        raise ValueError("Unsupported file extension. Please provide a valid FASTA or FASTQ file.")

def generate_label_dataframe(file_path):
    labels = OrderedDict()
    
    # Automatically detect file type
    file_type = detect_file_type(file_path)
    if file_type == 'fasta':
        seq_iter = fasta_iter(file_path)
    elif file_type == 'fastq':
        seq_iter = fastq_iter(file_path)
    else:
        raise ValueError("Unsupported file type. Use 'fasta' or 'fastq'.")

    for h, seq in seq_iter:
        # Ambil hanya ID sebelum karakter "|"
        seq_id = h.split('|')[0].strip()
        # Ekstrak label di antara tanda petik dua ("")
        label_match = re.search(r'"(.*?)"', h)
        label = label_match.group(1) if label_match else "Unknown"
        
        # Simpan ID dan label
        labels[seq_id] = label

    # Buat DataFrame dari dictionary labels
    label_df = pd.DataFrame.from_dict(labels, orient='index', columns=['Label'])
    
    return label_df

folder_path = './'
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.fna.gz')]

# Menggabungkan semua label dari beberapa file
label_df = pd.concat([generate_label_dataframe(f) for f in files])

# Reset indeks jika diperlukan
label_df.reset_index(inplace=True)
label_df.columns = ['Sequence_ID', 'Label']
label_df =  label_df[['Label']]
label_df

,Label
0,NC_006177.1 Symbiobacterium thermophilum IAM 1...
1,NC_006177.1 Symbiobacterium thermophilum IAM 1...
2,NC_006177.1 Symbiobacterium thermophilum IAM 1...
3,NC_006177.1 Symbiobacterium thermophilum IAM 1...
4,NC_006177.1 Symbiobacterium thermophilum IAM 1...
...,...
99995,"NC_007645.1 Hahella chejuensis KCTC 2396, comp..."
99996,"NC_007645.1 Hahella chejuensis KCTC 2396, comp..."
99997,"NC_007645.1 Hahella chejuensis KCTC 2396, comp..."
99998,"NC_007645.1 Hahella chejuensis KCTC 2396, comp..."


In [7]:
from sklearn.model_selection import train_test_split
import time
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [8]:
features = feature_df
labels = label_df

# Pisahkan data menjadi training dan testing set (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Jumlah data latih per kelas:")
print(np.unique(y_train, return_counts=True))

print("\nJumlah data uji per kelas:")
print(np.unique(y_test, return_counts=True))

Jumlah data latih per kelas:
(array(['NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome',
       'NC_003910.7 Colwellia psychrerythraea 34H, complete sequence',
       'NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence',
       'NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence',
       'NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome',
       'NC_008346.1 Syntrophomonas wolfei subsp. wolfei str. Goettingen G311, complete sequence',
       'NC_008709.1 Psychromonas ingrahamii 37, complete sequence',
       'NC_009922.1 Alkaliphilus oremlandii OhILAs, complete sequence',
       'NC_011830.1 Desulfitobacterium hafniense DCB-2, complete sequence',
       'NC_013422.1 Halothiobacillus neapolitanus c2, complete sequence'],
      dtype=object), array([8023, 7962, 7995, 8014, 8008, 8000, 7998, 7987, 8015, 7998]))

Jumlah data uji per kelas:
(array(['NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 c

In [9]:
# List model yang akan digunakan
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    #"Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    #"MLP Classifier": MLPClassifier(random_state=42, max_iter=500)
}

# Menyimpan hasil metrik dan waktu komputasi
results = []

# Loop melalui semua model dan evaluasi performanya
for model_name, model in models.items():
    start_time = time.time()  # Mulai penghitungan waktu
    
    # Latih model
    model.fit(X_train, y_train)
    
    # Prediksi menggunakan data testing
    y_pred = model.predict(X_test)
    
    # Hitung waktu komputasi
    elapsed_time = time.time() - start_time
    
    # Evaluasi model dengan berbagai metrik
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Simpan hasil dalam dictionary
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Time (seconds)": elapsed_time
    })
    
    # Tampilkan hasil classification report untuk setiap model
    print(f"\nClassification Report untuk {model_name}:")
    print(classification_report(y_test, y_pred))

# Tampilkan hasil dalam bentuk dataframe untuk mempermudah pembacaan
import pandas as pd
results_df = pd.DataFrame(results)
results_df

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Classification Report untuk Random Forest:
                                                                                         precision    recall  f1-score   support

                 NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.82      0.81      0.81      1977
                           NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.73      0.81      0.77      2038
                  NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.99      0.96      0.97      2005
                            NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.89      0.84      0.86      1986
  NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.81      0.91      0.86      1992
NC_008346.1 Syntrophomonas wolfei subsp. wolfei str. Goettingen G311, complete sequence       0.82      0.82      0.82      2000
                              NC_008709.1 Psychromon

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Classification Report untuk Logistic Regression:
                                                                                         precision    recall  f1-score   support

                 NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.79      0.83      0.81      1977
                           NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.72      0.78      0.75      2038
                  NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.98      0.97      0.98      2005
                            NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.88      0.83      0.86      1986
  NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.75      0.86      0.80      1992
NC_008346.1 Syntrophomonas wolfei subsp. wolfei str. Goettingen G311, complete sequence       0.81      0.75      0.78      2000
                              NC_008709.1 Psyc

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Classification Report untuk Support Vector Machine:
                                                                                         precision    recall  f1-score   support

                 NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.88      0.89      0.88      1977
                           NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.82      0.84      0.83      2038
                  NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.99      0.98      0.99      2005
                            NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.92      0.91      0.91      1986
  NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.89      0.92      0.91      1992
NC_008346.1 Syntrophomonas wolfei subsp. wolfei str. Goettingen G311, complete sequence       0.89      0.85      0.87      2000
                              NC_008709.1 P

/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



Classification Report untuk K-Nearest Neighbors:
                                                                                         precision    recall  f1-score   support

                 NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.77      0.86      0.82      1977
                           NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.68      0.81      0.74      2038
                  NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.99      0.98      0.98      2005
                            NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.90      0.84      0.87      1986
  NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.81      0.90      0.85      1992
NC_008346.1 Syntrophomonas wolfei subsp. wolfei str. Goettingen G311, complete sequence       0.79      0.81      0.80      2000
                              NC_008709.1 Psyc

,Model,Accuracy,Precision,Recall,F1 Score,Time (seconds)
0,Random Forest,0.82905,0.830539,0.82905,0.827812,87.355247
1,Logistic Regression,0.80900,0.810761,0.80900,0.807652,118.998418
2,Support Vector Machine,0.88400,0.884007,0.88400,0.883782,1428.956791
3,K-Nearest Neighbors,0.82390,0.826597,0.82390,0.822916,8.710609


In [10]:
# List model yang akan digunakan
models = {
    # "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    # "Logistic Regression": LogisticRegression(random_state=42),
    # "Support Vector Machine": SVC(random_state=42),
    # "K-Nearest Neighbors": KNeighborsClassifier(),
    # "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "MLP Classifier": MLPClassifier(random_state=42, max_iter=100)
}

# Menyimpan hasil metrik dan waktu komputasi
results = []

# Loop melalui semua model dan evaluasi performanya
for model_name, model in models.items():
    start_time = time.time()  # Mulai penghitungan waktu
    
    # Latih model
    model.fit(X_train, y_train)
    
    # Prediksi menggunakan data testing
    y_pred = model.predict(X_test)
    
    # Hitung waktu komputasi
    elapsed_time = time.time() - start_time
    
    # Evaluasi model dengan berbagai metrik
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Simpan hasil dalam dictionary
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Time (seconds)": elapsed_time
    })
    
    # Tampilkan hasil classification report untuk setiap model
    print(f"\nClassification Report untuk {model_name}:")
    print(classification_report(y_test, y_pred))

dff = pd.DataFrame(results)
dff

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(



Classification Report untuk MLP Classifier:
                                                                                         precision    recall  f1-score   support

                 NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.90      0.86      0.88      1977
                           NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.81      0.84      0.82      2038
                  NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.99      0.98      0.99      2005
                            NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.89      0.92      0.91      1986
  NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.89      0.92      0.90      1992
NC_008346.1 Syntrophomonas wolfei subsp. wolfei str. Goettingen G311, complete sequence       0.87      0.86      0.86      2000
                              NC_008709.1 Psychromo

,Model,Accuracy,Precision,Recall,F1 Score,Time (seconds)
0,MLP Classifier,0.8817,0.881698,0.8817,0.881512,1321.557557


In [11]:
hasil = pd.concat([results_df, dff], ignore_index=True)
hasil

,Model,Accuracy,Precision,Recall,F1 Score,Time (seconds)
0,Random Forest,0.82905,0.830539,0.82905,0.827812,87.355247
1,Logistic Regression,0.80900,0.810761,0.80900,0.807652,118.998418
2,Support Vector Machine,0.88400,0.884007,0.88400,0.883782,1428.956791
3,K-Nearest Neighbors,0.82390,0.826597,0.82390,0.822916,8.710609
4,MLP Classifier,0.88170,0.881698,0.88170,0.881512,1321.557557


# Reads

## Feature

In [10]:
read_file = './reads/anonymous_reads.fq.gz' 
sequences = file_iter(read_file)
patterns = [['101', '1111', '10001']]
for i, pattern_group in enumerate(patterns, start=1):
    kmer_counts = count_spaced_kmers(sequences, pattern_group)
    feature_name, feature_space = build_feature_space(kmer_counts, sequences)
    features = np.array(list(feature_space.values()))

feature_df = pd.DataFrame(features)
feature_df

Menghitung masing-masing fitur dengan paralelisme


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 666654/666654 [01:15<00:00, 8817.32it/s]


Membangun feature space


,0,1,2,3,4,5,6,7,8,9,...,278,279,280,281,282,283,284,285,286,287
0,0.026667,0.046667,0.073333,0.026667,0.040000,0.060000,0.040000,0.040000,0.00,0.006667,...,0.000000,0.000000,0.000000,0.006667,0.000000,0.000000,0.006667,0.000000,0.000000,0.000000
1,0.013333,0.066667,0.053333,0.020000,0.006667,0.073333,0.046667,0.026667,0.00,0.000000,...,0.006667,0.013333,0.000000,0.000000,0.006667,0.000000,0.000000,0.006667,0.000000,0.006667
2,0.000000,0.060000,0.073333,0.046667,0.033333,0.060000,0.060000,0.026667,0.00,0.006667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.046667,0.066667,0.073333,0.040000,0.053333,0.073333,0.040000,0.060000,0.00,0.000000,...,0.000000,0.000000,0.006667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.013333,0.033333,0.020000,0.020000,0.013333,0.040000,0.020000,0.020000,0.00,0.000000,...,0.013333,0.020000,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.006667,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666649,0.026667,0.060000,0.046667,0.033333,0.026667,0.020000,0.066667,0.053333,0.00,0.000000,...,0.000000,0.006667,0.000000,0.020000,0.013333,0.006667,0.000000,0.000000,0.013333,0.006667
666650,0.040000,0.046667,0.046667,0.040000,0.020000,0.080000,0.053333,0.020000,0.00,0.000000,...,0.006667,0.000000,0.000000,0.006667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
666651,0.046667,0.053333,0.066667,0.053333,0.046667,0.046667,0.100000,0.033333,0.00,0.006667,...,0.013333,0.000000,0.000000,0.000000,0.006667,0.000000,0.000000,0.006667,0.006667,0.006667
666652,0.033333,0.066667,0.066667,0.026667,0.060000,0.053333,0.053333,0.033333,0.00,0.000000,...,0.013333,0.006667,0.006667,0.006667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
labelR_path = "./reads/reads_mapping.tsv.gz"
labelR_df = pd.read_csv(labelR_path, sep='\t')
labelR_df = labelR_df[["genome_id"]]
labelR_df

,genome_id
0,Genome19.0
1,Genome19.0
2,Genome19.0
3,Genome19.0
4,Genome18.0
...,...
666649,Genome18.0
666650,Genome19.0
666651,Genome19.0
666652,Genome19.0


In [12]:
features = feature_df
labels = labelR_df

# Pisahkan data menjadi training dan testing set (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Jumlah data latih per kelas:")
print(np.unique(y_train, return_counts=True))

print("\nJumlah data uji per kelas:")
print(np.unique(y_test, return_counts=True))

Jumlah data latih per kelas:
(array(['Genome10.0', 'Genome10.1', 'Genome11.0', 'Genome12.0',
       'Genome13.0', 'Genome14.0', 'Genome15.0', 'Genome16.0',
       'Genome17.0', 'Genome18.0', 'Genome19.0', 'Genome2.0',
       'Genome20.0', 'Genome22.0', 'Genome23.0', 'Genome24.0',
       'Genome3.0', 'Genome4.0', 'Genome4.1', 'Genome5.0', 'Genome6.0',
       'Genome7.0', 'Genome8.0', 'Genome9.0'], dtype=object), array([  2843,   9214,     37,    438,   2081,   8187,   5417,    338,
         1936,  69103, 367831,  12720,    853,  13194,   2856,    444,
         9619,   5141,   1501,    340,   7383,    656,  10800,    391]))

Jumlah data uji per kelas:
(array(['Genome10.0', 'Genome10.1', 'Genome11.0', 'Genome12.0',
       'Genome13.0', 'Genome14.0', 'Genome15.0', 'Genome16.0',
       'Genome17.0', 'Genome18.0', 'Genome19.0', 'Genome2.0',
       'Genome20.0', 'Genome22.0', 'Genome23.0', 'Genome24.0',
       'Genome3.0', 'Genome4.0', 'Genome4.1', 'Genome5.0', 'Genome6.0',
       'Genome7.0'

In [13]:
# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Latih model
rf_model.fit(X_train, y_train)

# Prediksi menggunakan data testing
y_pred = rf_model.predict(X_test)

# Tampilkan hasil akurasi dan classification report
print("\nAkurasi Model:")
print(accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Akurasi Model:
0.6924271174745558

Classification Report:


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

  Genome10.0       0.00      0.00      0.00       759
  Genome10.1       0.00      0.00      0.00      2264
  Genome11.0       0.00      0.00      0.00         9
  Genome12.0       0.00      0.00      0.00       106
  Genome13.0       0.00      0.00      0.00       491
  Genome14.0       0.00      0.00      0.00      2001
  Genome15.0       0.00      0.00      0.00      1323
  Genome16.0       0.00      0.00      0.00        94
  Genome17.0       0.00      0.00      0.00       446
  Genome18.0       0.08      0.00      0.00     17141
  Genome19.0       0.69      1.00      0.82     92345
   Genome2.0       0.00      0.00      0.00      3170
  Genome20.0       0.00      0.00      0.00       189
  Genome22.0       0.00      0.00      0.00      3268
  Genome23.0       0.00      0.00      0.00       730
  Genome24.0       0.00      0.00      0.00       118
   Genome3.0       0.00      0.00      0.00      2399
   Genome4.0       0.00    

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
